
# 1 El análisis

In [3]:
import os
import winsound
import special_fun as s
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import librosa
import scipy.signal as signal
import pywt
from scipy.signal import firwin
#Para que me avise
duration = 1000 
freq = 440 

## 1.1 Carga obtención de los audios

In [4]:
ruta_carpeta = '../Data/audio_and_txt_files'
#Si es en colab
#ruta_carpeta = "/content/respiratory_sound_database/Respiratory_Sound_Database/audio_and_txt_files" 
lista_archivos = os.listdir(ruta_carpeta)
datos=s.sonido_probando123(lista_archivos,ruta_carpeta)

## Ejemplo del metodo usado

In [5]:
y,sr=librosa.load(list(datos.keys())[20])

In [7]:
high=100
low=2000
nyq=sr/2
bandalp=(low-high)*0.1
ordenlp=int(5*(nyq/bandalp))
if ordenlp%2==0:
    ordenlp+=1

low_pass=firwin(ordenlp, low/nyq ,window=('kaiser',7.85))

bandahp=(high)*0.1
ordenhp= int(5*(nyq/bandahp))
if ordenhp%2==0:
    ordenhp+=1

high_pass=firwin(ordenhp, high/nyq ,pass_zero="highpass",window=('kaiser',7.85))